## Trabajo Fin de Grado
### Gestor de Quirófanos
### Implementación de Heurística LPT+ para asignación de quirófanos

#### Autor: Jesús García Armario

In [27]:
# Imports
import numpy as np
import pandas as pd
from Utils import ActoQuirurgico, Quirofano

In [28]:
# Importamos el listado preprocesado de una especialidad para las pruebas
filename = '..\\..\\Datos\\Listados_Preprocesados\\ListadoInterv_Preprocesado_MAXILOFACIAL.xlsx'
df = pd.read_excel(filename)

In [29]:
# Seleccionamos una muestra aleatoria de 100 pacientes
df = df.sample(n=100, random_state=1)
# Nos quedamos sólo con el NHC, Ponderación y duración
df = df[['NHC', 'PONDERACIÓN', 'DURACIÓN']]

In [30]:
# Creamos un set de actos quirúrgicos
actos_pendientes = set()
i = 0
for elemento in df.itertuples():
    actos_pendientes.add(ActoQuirurgico(i, elemento[2], elemento[0], elemento[1]))
    i += 1


In [31]:
print ("Número de actos quirúrgicos: ", len(actos_pendientes))
def ordenaPacientes(actos_pendientes : set):
    # Calculamos los deciles de la ponderación
    deciles = np.percentile([acto.getPrioridad() for acto in actos_pendientes], np.arange(0, 100, 10))
    # Creamos un diccionario con los deciles y los pacientes que pertenecen a cada uno
    diccionario = {}
    for i in range(0, len(deciles)-1):
        diccionario[i] = set()
    for acto in actos_pendientes:
        for i in range(0, len(deciles)-1):
            if acto.getPrioridad() >= deciles[i] and acto.getPrioridad() < deciles[i+1]:
                diccionario[i].add(acto)
                break
    # Ordenamos los pacientes de cada decil por duración de mayor a menor
    for i in range(0, len(deciles)-1):
        diccionario[i] = sorted(diccionario[i], key=lambda acto: acto.getDuracion(), reverse=True)
    # Creamos una lista con los pacientes ordenados de mayor decil a menor
    actos_pendientes = []
    for i in range(len(diccionario)-1, -1, -1):
        actos_pendientes += diccionario[i]
    return actos_pendientes

actos_pendientes = ordenaPacientes(actos_pendientes)
print ("Número de actos quirúrgicos: ", len(actos_pendientes))
print("Pacientes ordenados por prioridad y duración: ")
for acto in actos_pendientes:
    print("Paciente: ", acto.getIdPaciente(), " Prioridad: ", acto.getPrioridad(), " Duración: ", acto.getDuracion())


Número de actos quirúrgicos:  100
Número de actos quirúrgicos:  90
Pacientes ordenados por prioridad y duración: 
Paciente:  2691  Prioridad:  3574487  Duración:  57
Paciente:  2750  Prioridad:  3588910  Duración:  35
Paciente:  2564  Prioridad:  3537939  Duración:  0
Paciente:  2698  Prioridad:  3575060  Duración:  0
Paciente:  2664  Prioridad:  3567265  Duración:  0
Paciente:  2563  Prioridad:  3537080  Duración:  0
Paciente:  2635  Prioridad:  3560457  Duración:  0
Paciente:  2805  Prioridad:  3600500  Duración:  0
Paciente:  2823  Prioridad:  3604122  Duración:  0
Paciente:  2738  Prioridad:  3585618  Duración:  0
Paciente:  2115  Prioridad:  3312452  Duración:  115
Paciente:  2135  Prioridad:  3324062  Duración:  68
Paciente:  2284  Prioridad:  3409369  Duración:  29
Paciente:  2542  Prioridad:  3529956  Duración:  28
Paciente:  2290  Prioridad:  3412840  Duración:  15
Paciente:  2226  Prioridad:  3380120  Duración:  0
Paciente:  2387  Prioridad:  3459096  Duración:  0
Paciente:  

In [32]:
# Función que asigna un acto quirúrgico a un quirófano siguiendo el criterio LTP
def LTP(actos_pendientes, n_quirofanos, dias, tiempo):
    # Primero ordenamos los actos quirúrgicos por duración y prioridad
    actos_pendientes = ordenaPacientes(actos_pendientes)
    # Creamos una lista de quirófanos, cada uno con su día y tiempo de trabajo
    quirofanos = [n_quirofanos][dias]
    for i in range(n_quirofanos):
        for j in range(dias):
            quirofanos[i][j] = Quirofano(i, j, tiempo)
    # Creamos una matriz de tiempos de trabajo
    tiempos = np.zeros((n_quirofanos, dias))